In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import pandas as pd
from keras_preprocessing.image import ImageDataGenerator
import tensorflow_addons as tfa

In [2]:
directory = 'C:\datasets\img_align_celeba\img_align_celeba'

In [3]:
tag = pd.read_csv('C:/datasets/list_attr_celeba.csv')

In [4]:
col = ['image_id', 'Male', 'Mustache', 'Bald', 'Smiling', 'Young']
tag = tag[col]
tag = tag.replace(-1, 0)
col = col[1:]
tag

,image_id,Male,Mustache,Bald,Smiling,Young
0,000001.jpg,0,0,0,1,1
1,000002.jpg,0,0,0,1,1
2,000003.jpg,1,0,0,0,1
3,000004.jpg,0,0,0,0,1
4,000005.jpg,0,0,0,0,1
...,...,...,...,...,...,...
202594,202595.jpg,0,0,0,0,1
202595,202596.jpg,1,0,0,1,1
202596,202597.jpg,1,0,0,1,1
202597,202598.jpg,0,0,0,1,1


In [5]:
train = tag.sample(frac = .8, random_state=42)
test = tag.drop(train.index)
train

,image_id,Male,Mustache,Bald,Smiling,Young
93241,093242.jpg,1,0,0,0,1
121413,121414.jpg,0,0,0,0,1
10624,010625.jpg,1,0,0,0,1
82523,082524.jpg,0,0,0,1,1
73192,073193.jpg,0,0,0,1,0
...,...,...,...,...,...,...
36890,036891.jpg,1,0,0,0,1
1992,001993.jpg,0,0,0,0,1
47852,047853.jpg,0,0,0,1,1
79821,079822.jpg,0,0,0,1,1


In [6]:
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)

In [7]:
train_generator=datagen.flow_from_dataframe(
dataframe=train,
directory = directory,
x_col="image_id",
y_col=col,
subset="training",
batch_size=16,
seed=42,
shuffle=True,
    class_mode='raw',
target_size=(256,256))

valid_generator=datagen.flow_from_dataframe(
dataframe=train,
directory=directory,
x_col="image_id",
y_col=col,
subset="validation",
batch_size=16,
seed=42,
shuffle=True,
class_mode="raw",
target_size=(256,256))

test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator=test_datagen.flow_from_dataframe(
dataframe=test,
directory=directory,
x_col="image_id",
y_col=col,
batch_size=16,
seed=42,
shuffle=False,
class_mode='raw',
target_size=(256,256))

Found 121560 validated image filenames.
Found 40519 validated image filenames.
Found 40520 validated image filenames.


In [9]:
class MLP(tf.keras.layers.Layer):
    def __init__(self, mlp1_dim, mlp2_dim):
        super(MLP, self).__init__()
        self.mlp1_dim = mlp1_dim
        self.mlp2_dim = mlp2_dim

        self.mlp1 = tf.keras.layers.Dense(self.mlp1_dim, kernel_initializer = tf.keras.initializers.lecun_normal())
        self.mlp2 = tf.keras.layers.Dense(self.mlp2_dim, kernel_initializer = tf.keras.initializers.lecun_normal())
        
    def call(self, X):
        X = self.mlp1(X)
        X = tf.nn.gelu(X)
        X = self.mlp2(X)
        return X

class Mixer_block(tf.keras.layers.Layer):
    def __init__(self, n_patches, hidden_chaneel_size, mlp_dim_S, mlp_dim_C):
        super(Mixer_block, self).__init__()
        self.n_patches = n_patches
        self.hidden_chaneel_size = hidden_chaneel_size
        self.mlp_dim_S = mlp_dim_S
        self.mlp_dim_C = mlp_dim_C
        
        self.ln = tf.keras.layers.LayerNormalization(axis = 1)
        self.t = tf.keras.layers.Permute((2,1))
        self.mlp1 = MLP(self.mlp_dim_S, self.n_patches)
        self.mlp2 = MLP(self.mlp_dim_C, self.hidden_chaneel_size)
        
    def call(self, X):
        skip_X = X
        X = self.ln(X)
        X = self.t(X)
        X = self.mlp1(X)
        X = self.t(X)
        X = X  + skip_X
        skip_X2 = X
        X = self.ln(X)
        X = self.mlp2(X)
        X = X + skip_X2
        return X

class MLP_Mixer(tf.keras.models.Model):
    def __init__(self, patch_size:int, image_size:int, hidden_channel_size:int, mlp_dim_S, mlp_dim_C, n_mixers:int, n_labels:int, mode:str = 'softmax'):
        tf.random.set_seed(42)
        super(MLP_Mixer, self).__init__()
        self.patch_size = patch_size
        self.image_size = image_size
        if (image_size % patch_size) != 0:
            raise ValueError('size error')
        self.n_patches = int((tf.square(self.image_size) / tf.square(self.patch_size)).numpy())
        self.hidden_channel_size = hidden_channel_size
        self.mlp_dim_S = mlp_dim_S
        self.mlp_dim_C = mlp_dim_C
        self.n_mixers = n_mixers
        self.n_labels = n_labels
        if mode not in ['sigmoid','softmax']:
            raise ValueError('mode must be sigmoid or softmax')
        else:
            self.mode = mode
        
        self.patchConv = tf.keras.layers.Conv2D(self.hidden_channel_size, (self.patch_size, self.patch_size), strides = (self.patch_size, self.patch_size))
        self.reshapeL = tf.keras.layers.Reshape((self.n_patches, self.hidden_channel_size))
        self.mixer_blocks = [Mixer_block(self.n_patches, self.hidden_channel_size, self.mlp_dim_S, self.mlp_dim_C) for _ in range(self.n_mixers)]
        self.gap = tf.keras.layers.GlobalAveragePooling1D()
        self.classifier = tf.keras.layers.Dense(1 if self.n_labels == 2 else self.n_labels, activation = self.mode, kernel_initializer = tf.keras.initializers.glorot_normal(seed = 42))       
            
    def call(self, X):
        X = self.patchConv(X)
        X = self.reshapeL(X)
        for mixer in self.mixer_blocks:
            X = mixer(X)
        X = self.gap(X)
        X = self.classifier(X)
        return X
    

In [13]:
mlp_mixer = MLP_Mixer(16, 256, 512, 256, 2048, 8, len(col), mode = 'sigmoid')
opt = tf.keras.optimizers.Adam(.00005)
mlp_mixer.compile(optimizer=opt, loss=tfa.losses.focal_loss.sigmoid_focal_crossentropy, metrics='acc')

In [14]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
mlp_mixer.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10)

Epoch 1/10
7597/7597 [==============================] - 785s 103ms/step - loss: 0.1473 - acc: 0.6995 - val_loss: 0.1183 - val_acc: 0.7342
Epoch 2/10
7597/7597 [==============================] - 758s 100ms/step - loss: 0.1111 - acc: 0.7696 - val_loss: 0.1097 - val_acc: 0.8089
Epoch 3/10
7597/7597 [==============================] - 793s 104ms/step - loss: 0.1016 - acc: 0.7831 - val_loss: 0.1060 - val_acc: 0.8335
Epoch 4/10
7597/7597 [==============================] - 792s 104ms/step - loss: 0.0954 - acc: 0.7944 - val_loss: 0.0991 - val_acc: 0.7821
Epoch 5/10
7597/7597 [==============================] - 798s 105ms/step - loss: 0.0905 - acc: 0.8022 - val_loss: 0.0950 - val_acc: 0.7847
Epoch 6/10
7597/7597 [==============================] - 793s 104ms/step - loss: 0.0860 - acc: 0.8087 - val_loss: 0.0948 - val_acc: 0.8057
Epoch 7/10
7597/7597 [==============================] - 795s 105ms/step - loss: 0.0819 - acc: 0.8160 - val_loss: 0.0943 - val_acc: 0.7944
Epoch 8/10
7597/7597 [============